In [116]:
import xml, os, pandas as pd, glob
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import fromstring, ElementTree
import requests, re
import time
import requests
import string
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

In [117]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
npp_ts = pd.read_csv('npp_timespans.csv', sep = ';')

In [118]:
os.chdir("D://Scriptie//Data//csv-metadata")
list_csv = glob.glob('*.csv')

In [119]:
def GetTS(newspaper_title):
    sy = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'sy'].iloc[0]
    ey = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'ey'].iloc[0]
    t = ey - sy
    return t

In [120]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [121]:
url1= "http://services.kb.nl/mdo/oai?verb=GetRecord&identifier=DDD:"
url3 = "&metadataPrefix=didl"

In [122]:
def DateXML(soup_element):
    t = str(soup_element.find_all("date")[0])[6:10]
    return t

In [123]:
def OCRXML(soup_element):
    t = soup_element.find_all("OCRConfidencelevel")
    t = [str(i) for i in t]
    t = [i.replace('<OCRConfidencelevel>', '') for i in t]
    t = [i.replace('<OCRConfidencelevel/>', '') for i in t]
    t = [i.replace('</OCRConfidencelevel>', '') for i in t]
    t = [i for i in t if i]
    t = [int(i.replace('0.', '')) for i in t]
    t = (sum(t) / len(t)) / 1000
    return t


In [124]:
def GetTitlesYearNpp(newspaper_title):
    fn = [fn for fn in list_csv if fn[0:4] == newspaper_title][0]
    
    df = pd.read_csv(fn, sep = '\t')
    df['id'] = df.id.str[:-6]
    pd.DataFrame(list(set(df.id)) ,columns = ['id'])
    
    old_npp_year = round(len(df) / GetTS(newspaper_title))
    
    if old_npp_year > 30:
        new_total = 30 * GetTS(newspaper_title)
        df = df.sample(n=new_total)
        df = df.reset_index(drop=True)
    
    titles_df = pd.DataFrame()
    
    for issue in list(df.id):
        url = url1 + issue + url3
        result = requests.get(url)
        content = result.content
        soup = BeautifulSoup(content, "xml")
        list_titles = soup.find_all("title")
        list_titles = [str(i) for i in list_titles]
        list_titles = [i.replace('<title>', '') for i in list_titles]
        list_titles = [i.replace('</title>', '') for i in list_titles]
        list_titles = [clean_and_split_str(i) for i in list_titles]
        
        for title in list_titles:
            t = pd.DataFrame([title], columns = ['title'])
            t['year'] = DateXML(soup)
            t['npp'] = newspaper_title
            t['ocr'] = OCRXML(soup)
            t['id'] = issue
            titles_df = titles_df.append(t)
    
    return titles_df
        
    

In [126]:
master_df = pd.DataFrame()

for ttl in tqdm([t[0:4] for t in list_csv]):
    master_df = master_df.append(GetTitlesYearNpp(ttl))



  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [129]:
GetTitlesYearNpp('brdc')

,title,year,npp,ocr,id
0,bredasche courant,1832,brdc,0.80150,ddd:010243015:mpeg21
0,nederlanden breda den 6den april,1832,brdc,0.80150,ddd:010243015:mpeg21
0,dagorder,1832,brdc,0.80150,ddd:010243015:mpeg21
0,dagorder,1832,brdc,0.80150,ddd:010243015:mpeg21
0,belgie brussel den 5den april,1832,brdc,0.80150,ddd:010243015:mpeg21
0,frankrijk parijs den 3den april,1832,brdc,0.80150,ddd:010243015:mpeg21
0,beursprijs der effecten te amsterdam,1832,brdc,0.80150,ddd:010243015:mpeg21
0,advertentie,1832,brdc,0.80150,ddd:010243015:mpeg21
0,nederlanden breda den 6den april,1832,brdc,0.80150,ddd:010243015:mpeg21
0,dagorder,1832,brdc,0.80150,ddd:010243015:mpeg21
